# 课题实践二 User-CF 基于用户的协同过滤算法
**先找到相似用户，再找到他们喜欢的物品**，基于用户的协同过滤算法一般分为两个步骤：  
* (1) 找到和目标用户兴趣相似的用户集合。
* (2) 找到这个集合中的用户喜欢的，且目标用户没有听说过的物品推荐给目标用户。

> 备注：**以下代码为了直观地说明理论，并不是最优做法，有兴趣的同学可自行查看相关书籍。**

## 计算用户的兴趣相似度
在这里，我们用用户的行为相似度来计算其兴趣相似度。  
给定用户u和用户v，令N(u)表示用户u曾经有过购买的物品集合，令N(v) 为用户v曾经有过购买的物品集合。那么，我们可以通过如下的余弦相似度公式简单地计算u和v的兴趣相似度：  
$$ w_{uv} = \frac{|N(u)\bigcap{N(v)}|}{\sqrt{|N(u)||{N(v)}|}} $$

## 加载原始数据并整理

In [1]:
import pandas as pd

events = pd.read_csv("./events.csv")
print(events.dtypes)

# 填充空值
events = events.fillna(0)

# 转换transactionid列为整型
events["transactionid"] = events["transactionid"].astype("int")
events["event"] = events["event"].astype("str")

# 增加一列，显示可读时间
events["action_time"] = pd.to_datetime(events["timestamp"], unit='ms')

# 展示开头的20个数据行
events.head(20)

timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object


,timestamp,visitorid,event,itemid,transactionid,action_time
0,1433221332117,257597,view,355908,0,2015-06-02 05:02:12.117
1,1433224214164,992329,view,248676,0,2015-06-02 05:50:14.164
2,1433221999827,111016,view,318965,0,2015-06-02 05:13:19.827
3,1433221955914,483717,view,253185,0,2015-06-02 05:12:35.914
4,1433221337106,951259,view,367447,0,2015-06-02 05:02:17.106
5,1433224086234,972639,view,22556,0,2015-06-02 05:48:06.234
6,1433221923240,810725,view,443030,0,2015-06-02 05:12:03.240
7,1433223291897,794181,view,439202,0,2015-06-02 05:34:51.897
8,1433220899221,824915,view,428805,0,2015-06-02 04:54:59.221
9,1433221204592,339335,view,82389,0,2015-06-02 05:00:04.592


## 获取购买数据

In [13]:
import time

# 从日期字符串获取时间戳
def get_timestamp(dstr):
    timeArray = time.strptime(dstr, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray)) * 1000
    return timeStamp
    
start_time_str = "2015-09-01 00:00:00"
start_timestamp = get_timestamp(start_time_str)
print(start_timestamp)

print(events.shape[0])

# 只保留有购买行为的记录，且取时间为2015-09-01 00:00:00以后的数据
events = events.loc[(events["event"] == "transaction") & (events["timestamp"] > start_timestamp)]
print(events.shape[0])

events.head(10)

1441036800000
2478
2478


,timestamp,visitorid,event,itemid,transactionid,action_time
1146002,1441083703528,177211,transaction,301721,16851,2015-09-01 05:01:43.528
1146240,1441054223305,20984,transaction,84595,5755,2015-08-31 20:50:23.305
1146303,1441054485329,990624,transaction,35158,15782,2015-08-31 20:54:45.329
1146328,1441056112559,1210136,transaction,358927,16706,2015-08-31 21:21:52.559
1146370,1441064300239,1318167,transaction,404615,10277,2015-08-31 23:38:20.239
1146487,1441062687729,645525,transaction,355994,3320,2015-08-31 23:11:27.729
1147071,1441080169594,177211,transaction,301721,12474,2015-09-01 04:02:49.594
1147327,1441043697659,569539,transaction,290999,113,2015-08-31 17:54:57.659
1147374,1441045186227,530559,transaction,400334,17356,2015-08-31 18:19:46.227
1147391,1441044690075,991189,transaction,167121,1534,2015-08-31 18:11:30.075


## 获取访问者及其购买过的物品集合

In [13]:
# 获取某个用户购买过的物品列表
def get_items(event_group):
    return event_group.to_list()

visitors = events.groupby("visitorid").agg(items=("itemid", get_items)).reset_index()
print(visitors.shape[0])
visitors.head(20)

1344


,visitorid,items
0,264,"[459835, 161949]"
1,2519,[413125]
2,2807,[240440]
3,3048,[331032]
4,3581,[220793]
5,4537,"[166106, 166106]"
6,5207,[37174]
7,5939,[379294]
8,6750,[461190]
9,6952,"[461686, 108924, 19789]"


## 计算用户之间的兴趣相似度

In [15]:
# 自相交笛卡尔积
t1 = time.time()

visitors1 = visitors.rename(columns={"visitorid":"visitorid1", "items":"items1"})

# 为了计算笛卡尔积，两个dataframe都增加一个常数列key
visitors["key"] = 0
visitors1["key"] = 0

# 笛卡尔积
visitor_sims = visitors.merge(visitors1, on="key")
print(visitor_sims.shape[0])
visitor_sims = visitor_sims.loc[visitor_sims.visitorid < visitor_sims.visitorid1]
print(visitor_sims.shape[0])

t2 = time.time()
print(f"it takes: {t2 - t1: .5f}s")

visitor_sims.head(10)

1806336
902496
it takes:  0.29523s


,visitorid,items,key,visitorid1,items1
1,264,"[459835, 161949]",0,2519,[413125]
2,264,"[459835, 161949]",0,2807,[240440]
3,264,"[459835, 161949]",0,3048,[331032]
4,264,"[459835, 161949]",0,3581,[220793]
5,264,"[459835, 161949]",0,4537,"[166106, 166106]"
6,264,"[459835, 161949]",0,5207,[37174]
7,264,"[459835, 161949]",0,5939,[379294]
8,264,"[459835, 161949]",0,6750,[461190]
9,264,"[459835, 161949]",0,6952,"[461686, 108924, 19789]"
10,264,"[459835, 161949]",0,7633,[7943]


In [16]:
import math 

# 计算用户之间的相似度得分
def calculate_similarity(df):
    count_up = len(set(df["items"]).intersection(set(df["items1"])))
    count_down = math.sqrt(len(set(df["items"]))*len(set(df["items1"])))
    return count_up/count_down

t1 = time.time()

visitor_sims["sim_score"] = visitor_sims.apply(calculate_similarity, axis=1)

t2 = time.time()
print(f"it takes: {t2 - t1: .5f}s")

it takes:  80.76127s


## 过滤得分为0的数据

In [17]:
print(visitor_sims.shape[0])
visitor_sims = visitor_sims[visitor_sims.sim_score > 0]
print(visitor_sims.shape[0])
visitor_sims.head(10)

902496
1217


,visitorid,items,key,visitorid1,items1,sim_score
1444,2519,[413125],0,102454,[413125],1.000000
4283,3048,[331032],0,264847,[331032],1.000000
7189,4537,"[166106, 166106]",0,508655,[166106],1.000000
10799,6750,[461190],0,47803,"[461190, 315543]",0.707107
10990,6750,[461190],0,247235,"[85914, 305798, 313723, 8588, 334520, 80299, 2...",0.137361
11242,6750,[461190],0,530559,"[400334, 235559, 356475, 253156, 384302, 24175...",0.204124
12162,6952,"[461686, 108924, 19789]",0,71586,[461686],0.577350
12250,6952,"[461686, 108924, 19789]",0,152963,"[176995, 152120, 302959, 92361, 20010, 76121, ...",0.055815
12279,6952,"[461686, 108924, 19789]",0,173863,"[241853, 33087, 268755, 233132, 151125, 218126...",0.160128
12372,6952,"[461686, 108924, 19789]",0,303083,"[10572, 461686]",0.408248


# 整理列并补全另一半数据，然后按照得分高低排序

In [18]:
visitor_sims1 = visitor_sims[["visitorid", "visitorid1", "sim_score"]]
visitor_sims2 = visitor_sims1.rename(columns={"visitorid":"visitorid1", "visitorid1":"visitorid"})[["visitorid", "visitorid1", "sim_score"]]
full_visitor_sims = pd.concat([visitor_sims1,visitor_sims2])

# 按照相似度得分排序
full_visitor_sims = full_visitor_sims.sort_values(["visitorid", "sim_score"], ascending=False)
print(full_visitor_sims.shape[0])

full_visitor_sims.head(20)

2434


,visitorid,visitorid1,sim_score
1123583,1406708,891134,1.000000
799676,1404265,642233,0.707107
736506,1399705,602801,1.000000
659897,1399500,530559,0.204124
807737,1399500,645525,0.174078
1326506,1384460,1045440,1.000000
1020071,1382772,816637,1.000000
1459559,1382772,1150086,0.129099
1753894,1382092,1370169,1.000000
208294,1382092,152963,0.096674


## 对每个用户生成top5相似的用户列表

In [19]:
# 获取某用户兴趣最相似的五个用户
def get_sims_top5(visitor_group):
    return visitor_group.to_list()[:5]

def get_score_top5(visitor_group):
    return visitor_group.to_list()[:5]

visitor_sims_top5 = full_visitor_sims.groupby("visitorid").agg(sim_list=("visitorid1", get_sims_top5), score_list=("sim_score", get_score_top5)).reset_index()
print(visitor_sims_top5.shape[0])

visitor_sims_top5.head(20)

500


,visitorid,sim_list,score_list
0,2519,[102454],[1.0]
1,3048,[264847],[1.0]
2,4537,[508655],[1.0]
3,6750,"[47803, 530559, 247235]","[0.7071067811865475, 0.20412414523193154, 0.13..."
4,6952,"[71586, 510429, 603999, 642130, 747952]","[0.5773502691896258, 0.5773502691896258, 0.577..."
5,7633,"[148244, 466583, 1054695, 1199182, 727576]","[1.0, 1.0, 1.0, 1.0, 0.5]"
6,17970,"[956435, 1125765, 1150086]","[1.0, 1.0, 0.12909944487358055]"
7,24688,"[1090325, 619488]","[1.0, 0.7071067811865475]"
8,25809,"[883404, 1194554]","[1.0, 1.0]"
9,28387,[198270],[0.09950371902099892]


## 计算用户对某物品的感兴趣程度
得到用户兴趣相似的用户后，User-CF算法会给用户推荐和他兴趣最相似的K个用户喜欢的物品。如下的公式度量了User-CF算法中用户u对物品i的感兴趣程度：  
$$p(u,i) = \sum_{v\in{S(u,K)\bigcap{N(i)}}}w_{uv}r_{vi}$$  
其中，$S(u,K)$包含和用户u兴趣最接近的K个用户，$N(i)$是对物品i有过行为的用户集合，$W_{uv}$是用户u和用户v的兴趣相似度，$r_{vi}$代表用户v对物品i的兴趣，因为对于物品i用户有过购买行为，所以所有的$r_{vi}$可认为是1。

### 获取2015-09-01后某物品的购买者列表

In [20]:
# 获取购买过某物品的用户列表
def get_visitors(event_group):
    return event_group.to_list()

items = events.groupby("itemid").agg(visitors=("visitorid", get_visitors)).reset_index()
print(items.shape[0])
items.head(20)

1912


,itemid,visitors
0,147,"[582525, 582525, 582525]"
1,168,[152963]
2,304,[1104446]
3,698,[669989]
4,835,[1127962]
5,856,[954276]
6,963,[30687]
7,1152,[214479]
8,1261,[390258]
9,1377,[935726]


### 将上述物品列表和用户相似度列表做笛卡尔积

In [21]:
t1 = time.time()

items["key"] = 0
visitor_sims_top5["key"] = 0

visitors_and_items = items.merge(visitor_sims_top5, on="key")
print(visitors_and_items.shape[0])

t2 = time.time()
print(f"it takes: {t2 - t1: .5f}s")

visitors_and_items.head(10)

956000
it takes:  0.10412s


,itemid,visitors,key,visitorid,sim_list,score_list
0,147,"[582525, 582525, 582525]",0,2519,[102454],[1.0]
1,147,"[582525, 582525, 582525]",0,3048,[264847],[1.0]
2,147,"[582525, 582525, 582525]",0,4537,[508655],[1.0]
3,147,"[582525, 582525, 582525]",0,6750,"[47803, 530559, 247235]","[0.7071067811865475, 0.20412414523193154, 0.13..."
4,147,"[582525, 582525, 582525]",0,6952,"[71586, 510429, 603999, 642130, 747952]","[0.5773502691896258, 0.5773502691896258, 0.577..."
5,147,"[582525, 582525, 582525]",0,7633,"[148244, 466583, 1054695, 1199182, 727576]","[1.0, 1.0, 1.0, 1.0, 0.5]"
6,147,"[582525, 582525, 582525]",0,17970,"[956435, 1125765, 1150086]","[1.0, 1.0, 0.12909944487358055]"
7,147,"[582525, 582525, 582525]",0,24688,"[1090325, 619488]","[1.0, 0.7071067811865475]"
8,147,"[582525, 582525, 582525]",0,25809,"[883404, 1194554]","[1.0, 1.0]"
9,147,"[582525, 582525, 582525]",0,28387,[198270],[0.09950371902099892]


### 计算用户对某物品的兴趣得分

In [23]:
# 计算用户对某个物品的喜好分
def calculate_score(df):
    sim_list = df["sim_list"]
    score_list = df["score_list"]
    item_visitors = df["visitors"]
    list_len = len(sim_list)
    amount = 0.0
    for i in range(list_len):
        visitor = sim_list[i]
        if visitor in item_visitors:
            amount += score_list[i]
    return amount

t1 = time.time() 

visitors_and_items["score"] = visitors_and_items.apply(calculate_score, axis=1)
visitors_and_items = visitors_and_items.loc[visitors_and_items["score"] > 0.0]
print(visitors_and_items.shape[0])

t2 = time.time()
print(f"it takes: {t2 - t1: .5f}s")

visitors_and_items.head(10)

10637
it takes:  65.42493s


,itemid,visitors,key,visitorid,sim_list,score_list,score
531,168,[152963],0,84494,[152963],[0.09667364890456635],0.096674
596,168,[152963],0,255918,"[256429, 214479, 198270, 152963]","[0.35355339059327373, 0.22360679774997896, 0.0...",0.048337
624,168,[152963],0,343283,"[749501, 469022, 152963]","[1.0, 0.2773500981126146, 0.09667364890456635]",0.096674
649,168,[152963],0,393947,"[565892, 1263358, 152963]","[1.0, 1.0, 0.09667364890456635]",0.096674
665,168,[152963],0,445245,"[878466, 152963]","[0.7071067811865475, 0.09667364890456635]",0.096674
691,168,[152963],0,512033,[152963],[0.06835859270246633],0.068359
692,168,[152963],0,513361,[152963],[0.09667364890456635],0.096674
693,168,[152963],0,518659,"[646913, 361291, 451072, 530559, 152963]","[0.35355339059327373, 0.35355339059327373, 0.3...",0.034179
709,168,[152963],0,565892,"[1263358, 393947, 152963]","[1.0, 1.0, 0.09667364890456635]",0.096674
721,168,[152963],0,614077,"[642124, 1001317, 152963]","[0.7071067811865475, 0.7071067811865475, 0.068...",0.068359


### 按照得分高低排序

In [25]:
# 移除不需要的列
final_scores = visitors_and_items[["visitorid", "itemid", "score"]]

# 按照得分高低排序
final_scores = final_scores.sort_values(["visitorid", "score"], ascending=False)
final_scores.head(40)

,visitorid,itemid,score
423499,1406708,209184,1.000000
863998,1404265,424912,0.707107
20997,1399705,8833,1.000000
243996,1399500,119736,0.378202
695496,1399500,338660,0.378202
38496,1399500,17359,0.204124
121996,1399500,63543,0.204124
217496,1399500,108343,0.204124
270496,1399500,132338,0.204124
323496,1399500,157150,0.204124
